#### STEP 1 : IMPORT ALL MODULES

In [136]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd

#### STEP 2 : MAKE / VISUALIZE DATASET

In [137]:
dataobj = pd.read_csv("resale-flat-prices/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")
housingDF = pd.DataFrame(dataobj)
housingDF.columns = [x.replace("_", " ") for x in housingDF.columns]
housingDF.head(5)

,month,town,flat type,block,street name,storey range,floor area sqm,flat model,lease commence date,remaining lease,resale price
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,10 TO 12,44.0,Improved,1979,61 years 04 months,232000.0
1,2017-01,ANG MO KIO,3 ROOM,108,ANG MO KIO AVE 4,01 TO 03,67.0,New Generation,1978,60 years 07 months,250000.0
2,2017-01,ANG MO KIO,3 ROOM,602,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,262000.0
3,2017-01,ANG MO KIO,3 ROOM,465,ANG MO KIO AVE 10,04 TO 06,68.0,New Generation,1980,62 years 01 month,265000.0
4,2017-01,ANG MO KIO,3 ROOM,601,ANG MO KIO AVE 5,01 TO 03,67.0,New Generation,1980,62 years 05 months,265000.0


#### STEP 3 : MEASURE OUR WORKING DATASET

In [138]:
for x in housingDF.columns:
    print(housingDF[x].nunique())
housingDF.shape

39
26
7
2409
543
17
160
20
51
616
2329


(70103, 11)

#### STEP 4 : IMPLEMENT ML MODEL

In [139]:
allvalues = dataobj.values
housingdata = allvalues[:, :4]
housingtarget = allvalues[:, -1].astype("float")
xtrain, xtest, ytrain, ytest = train_test_split(housingdata, housingtarget, random_state = 8)
encoderobj = OneHotEncoder(sparse = False)
xtrainencoded = encoderobj.fit_transform(xtrain[:5000]).astype("float")
xtrainencoded.shape
rfrobj = RandomForestRegressor(n_estimators = 10, random_state = 8)
rfrobj.fit(xtrainencoded, ytrain[:5000])
print(rfrobj.score(xtrainencoded, ytrain[:5000]))

0.9581113447408307


#### STEP 5 : CALCULATE ACCURACY

#### STEP 6 : PREDICT VALUE